# Summary
This notebook performs a sentiment analysis on a database of relevant comments from the regulations.gov Title IX public request for comments and analyzes the results. 
The average sentiment is calculated for the original dataset and the dataset without duplicate comments. The comment with the closest sentiment to the average of each dataset is displayed for reference.

The sentiment analysis code library used is TextBlob.

In [26]:
import numpy as np
import pandas
import re
from textblob import TextBlob

The comments are loaded in from the Data Cleanup notebook.

In [27]:
data = pandas.read_json('./data/comments_cleaned.json', orient='records', dtype='false')
data_duplicates_removed = pandas.read_json('./data/comments_duplicates_removed.json', orient='records', dtype='false')

Below is a print out of the pandas dataframe schema and first comment of the original and duplicates removed datasets.

In [28]:
print("Loaded %d comments." % len(data))
display(data[:1])

print("Loaded %d comments." % len(data_duplicates_removed))
display(data_duplicates_removed[:1])

Loaded 12225 comments.


,doc.attachment_download,doc.attachment_download -href,doc.attachment_name,doc.category,doc.city,doc.comment_body,doc.country,doc.name,doc.state,doc.zip
0,None,None,None,None,United States,"Dear Assistant General Counsel Hilary Malawer,...",Parent/Relative,Heather Hirsch,MN,55016


Loaded 2868 comments.


,doc.attachment_download,doc.attachment_download -href,doc.attachment_name,doc.category,doc.city,doc.comment_body,doc.country,doc.name,doc.state,doc.zip
0,None,None,None,None,United States,"Dear Assistant General Counsel Hilary Malawer,...",Individual,Emily McDonald,KS,67215


Remove links and special characters from the comment body.

In [29]:
def remove_links(comment):
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", str(comment)).split())

data['doc.comment_body'] = data['doc.comment_body'].map(remove_links)
data_duplicates_removed['doc.comment_body'] = data_duplicates_removed['doc.comment_body'].map(remove_links)

Calculating the sentiment score of each comment.

In [30]:
%%time
data['sentiment'] = data.apply(lambda x: TextBlob(str(x['doc.comment_body'])).sentiment.polarity, axis=1)
data_duplicates_removed['sentiment'] = data_duplicates_removed.apply(lambda x: TextBlob(str(x['doc.comment_body'])).sentiment.polarity, axis=1)

CPU times: user 50.8 s, sys: 703 ms, total: 51.5 s
Wall time: 54.5 s


A sample of comments from each dataset and their sentiment scores.

In [31]:
with pandas.option_context('display.max_columns', 14, 'display.max_colwidth', 5000):
    display(data[['doc.comment_body', 'sentiment']].sample(100))

,doc.comment_body,sentiment
11743,Dear Assistant General Counsel Hilary Malawer We cannot go backwards too many young people need and expect this for themselves and their family members and friends All Department of Education civil rights regulations and guidance documents are important and necessary Far from being burdensome current civil rights rules and regulations benefit schools and students by providing a clear framework that when followed allow all students an equal opportunity to learn in a safe and welcoming environment regardless of sex race color national origin disability status English proficiency sexual orientation or gender identity I urge the Department to keep in its current form 34 C F R pts 1 thru 1299 which include regulations governing the Secretary and the offices for Civil Rights Elementary and Secondary Education Special Education and Rehabilitative Services Career Technical and Adult Education Post Secondary Education Educational Research and Improvement and the National Council on Disability I also urge the Department to preserve all current significant guidance documents including guidance on sexual racial and disability based harassment including guidance on sexual violence access to athletic opportunities gender equity in career and technical education single sex schools equal access to educational resources nondiscriminatory school discipline racial diversity programs the rights of students with disabilities in charter schools restraint and seclusion of students with disabilities and the rights of English language learners I urge you to keep current regulations and guidance in place and to continue enforcing these critical civil rights laws so that all students have an equal opportunity to learn and thrive Sincerely Diane Milutinovich Fresno CA 93711,0.142749
3501,Dear Assistant General Counsel Hilary Malawer All Department of Education civil rights regulations and guidance documents are important and necessary Far from being burdensome current civil rights rules and regulations benefit schools and students by providing a clear framework that when followed allow all students an equal opportunity to learn in a safe and welcoming environment regardless of sex race color national origin disability status English proficiency sexual orientation or gender identity I urge the Department to keep in its current form 34 C F R pts 1 thru 1299 which include regulations governing the Secretary and the offices for Civil Rights Elementary and Secondary Education Special Education and Rehabilitative Services Career Technical and Adult Education Post Secondary Education Educational Research and Improvement and the National Council on Disability I also urge the Department to preserve all current significant guidance documents including guidance on sexual racial and disability based harassment including guidance on sexual violence access to athletic opportunities gender equity in career and technical education single sex schools equal access to educational resources nondiscriminatory school discipline racial diversity programs the rights of students with disabilities in charter schools restraint and seclusion of students with disabilities and the rights of English language learners I urge you to keep current regulations and guidance in place and to continue enforcing these critical civil rights laws so that all students have an equal opportunity to learn and thrive Sincerely Valeria Spinner Banks Milwaukee WI 53224,0.132604
11695,Dear Assistant General Counsel Hilary Malawer All Department of Education civil rights regulations and guidance documents are important and necessary Far from being burdensome current civil rights rules and regulations benefit schools and students by providing a clear framework that when followed allow all students an equal opportunity to learn in a safe and welcoming environment regardless of sex race color national origin disability status English proficiency sexual orientation or gender 

In [32]:
with pandas.option_context('display.max_columns', 14, 'display.max_colwidth', 5000):
    display(data_duplicates_removed[['doc.comment_body', 'sentiment']].sample(100))

,doc.comment_body,sentiment
1486,So clearly you don t believe in equal opportunity All men are created equal but they didn t really mean that and they didn t say women Women should have to work twice as hard as men for the same goal obviously otherwise how would it be fair And Black women should have to work ten times harder and potentially be falsely arrested for being black on campus otherwise those poor white precious egos might be threatened LGBTQ students well they just don t deserve education at all do they In fact if you aren t a prosperity gospel Christian white man you don t deserve anything at all Equality Ha That s only for rich white men That s right I forgot rich before OK that s fixed The only people who deserve an education are rich white men who can pretend to a brand of pseudo Christianity And what about everyone else The only education they deserve is to be raped and murdered for their gall and impudence in existing and trying to get an education Equality Never Right That s what DeVos believes and what she wants to implement As she d realize if she d ever had a real education at a real school,0.058496
987,To Whom It May Concern I am commenting on Regulation ID ED 2015 OSERS 001 1167 State Vocational Rehabilitation Services Program State Supported Employment Services Program Limitations on Use of Sub minimum Wage They need to be able to make a choice of being able to work in a sub minimum wage if this is taken out that could limit them on working on there choice job and limiting them from doing a job that suits them,0.433333
1427,I am a former college and national level athlete and now an Athletic Director at a girls school in Massachusetts Sports provide its participants with lifelong health education and leadership benefits equally important to our daughters and our sons I urge you to keep Title IX strong by preserving the current rules and enforcing this crucial legislation,0.122222
2388,Dear Assistant General Counsel Hilary Malawer I am writing to support the civil rights regulations and guidance in place in the Department of Education since the 1960s as part of our country s efforts to extend equal opportunity to students and communities that have traditionally been and continue to be discriminated against Far from being burdensome current civil rights rules and regulations benefit schools and students by providing a clear framework that when followed allow all students an equal opportunity to learn in a safe and welcoming environment regardless of sex race color national origin disability status English proficiency sexual orientation or gender identity I urge the Department to keep in its current form 34 C F R pts 1 thru 1299 which include regulations governing the Secretary and the offices for Civil Rights Elementary and Secondary Education Special Education and Rehabilitative Services Career Technical and Adult Education Post Secondary Education Educational Research and Improvement and the National Council on Disability I also urge the Department to preserve all current significant guidance documents including guidance on sexual racial and disability based harassment including guidance on sexual violence access to athletic opportunities gender equity in career and technical education single sex schools equal access to educational resources nondiscriminatory school discipline racial diversity programs the rights of students with disabilities in charter schools restraint and seclusion of students with disabilities and the rights of English language learners I urge you to keep current regulations and guidance in place and to continue enforcing these critical civil rights laws so that all students have an equal opportunity to learn and thrive Sincerely Joan Siegwald Mobile AL 36609,0.119700
2599,We must maintain our civil and human rights We will not go back,0.000000
2454,Subject WIOA In accordance with Executive Order 13777 Enforcing the Regulatory Reform Agenda the dept of Education is seeking input on r

Calculating the average sentiment for all comments.

In [33]:
original_avg_sentiment = data['sentiment'].sum() / len(data.index)
dup_removed_avg_sentiment = data_duplicates_removed['sentiment'].sum() / len(data_duplicates_removed.index)

print('Average sentiment of original dataset: %.10f.' % original_avg_sentiment)
print('Average sentiment of dataset with duplicates removed: %.10f.' % dup_removed_avg_sentiment)

Average sentiment of original dataset: 0.1400468345.
Average sentiment of dataset with duplicates removed: 0.1585561913.


Displaying the comment that most closely matches the average sentiment.

In [34]:
original_closest_sentiment = (np.abs(data['sentiment'] - original_avg_sentiment)).idxmin()
dup_removed_closest_sentiment = (np.abs(data_duplicates_removed['sentiment'] - dup_removed_avg_sentiment)).idxmin()

with pandas.option_context('display.max_colwidth', 5000):
    display(data.iloc[[original_closest_sentiment]][['doc.comment_body']])
    
with pandas.option_context('display.max_colwidth', 5000):
    display(data.iloc[[dup_removed_closest_sentiment]][['doc.comment_body']])

,doc.comment_body
7945,These 10 constituents representing U S Territories and military personnel join with the American Association of University Women AAUW in urging the U S Department of Education to protect Title IX preserve all of its current regulations and guidance and fully enforce the law Title IX of the Education Amendments of 1972 is the federal law that prohibits sex discrimination in education This vital law affects all areas of education It requires recipients of federal education funding to evaluate their current policies and practices adopt and publish a policy against sex discrimination and implement grievance procedures providing for prompt and equitable resolution of student and employee discrimination complaints Unfortunately many students still do not have access to an equitable education free from sex discrimination As Secretary of Education you have the power to address this critical civil rights issue and help make schools safer and more equitable for all students These 10 individuals join with AAUW in urging the U S Department of Education to keep in place current Title IX guidance and regulations and fully enforce the law


,doc.comment_body
1766,Dear Assistant General Counsel Hilary Malawer All Department of Education civil rights regulations and guidance documents are important and necessary Far from being burdensome current civil rights rules and regulations benefit schools and students by providing a clear framework that when followed allow all students an equal opportunity to learn in a safe and welcoming environment regardless of sex race color national origin disability status English proficiency sexual orientation or gender identity I urge the Department to keep in its current form 34 C F R pts 1 thru 1299 which include regulations governing the Secretary and the offices for Civil Rights Elementary and Secondary Education Special Education and Rehabilitative Services Career Technical and Adult Education Post Secondary Education Educational Research and Improvement and the National Council on Disability I also urge the Department to preserve all current significant guidance documents including guidance on sexual racial and disability based harassment including guidance on sexual violence access to athletic opportunities gender equity in career and technical education single sex schools equal access to educational resources nondiscriminatory school discipline racial diversity programs the rights of students with disabilities in charter schools restraint and seclusion of students with disabilities and the rights of English language learners I urge you to keep current regulations and guidance in place and to continue enforcing these critical civil rights laws so that all students have an equal opportunity to learn and thrive Sincerely Aaron Sterr Port Chester NY 10573


Resources: 
https://streamhacker.com/2010/05/10/text-classification-sentiment-analysis-naive-bayes-classifier/
https://web.stanford.edu/~jurafsky/slp3/6.pdf
